In [2]:
import torch

from utils import load_epsilon_net, load_image
from utils import load_epsilon_net
from sampling.dps import dps, dps_save
from sampling.dps_dpms import dps_dpms_save
from sampling.dmps import dpms_save
from time import time
import os
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

device = "cuda:0"
#n_steps = 100
torch.set_default_device(device)
seed = 2024
torch.manual_seed(seed=seed) # for reproducibility


# load the image
img_path = "./hackathon_starter_kit/material/celebahq_img/00010.jpg"
x_origin = load_image(img_path, device)


# load the degradation operator
#path_operator = f"./material/degradation_operators/sr16.pt"
path_operator = f"./hackathon_starter_kit/material/degradation_operators/outpainting_half.pt"
degradation_operator = torch.load(path_operator, map_location=device)

# apply degradation operator
y = degradation_operator.H(x_origin[None])
y = y.squeeze(0)

sigma = [0.01]
n_steps = [100]


method = "dpms"

output_base= "./output/"



for n in n_steps:
    for s in sigma:
        if method == "dps_dpms":
            K = [int(n/10)]
        else:
            K = [0]
        for k in K:

            # add noise
            y = y + s * torch.randn_like(y)

            # define inverse problem
            inverse_problem = (y, degradation_operator, s)

            # load model
            eps_net = load_epsilon_net("celebahq", n, device)

            # solve problem
            initial_noise = torch.randn((1, 3, 256, 256), device=device)


            #make output dir


            if method == "dps":
                output_dir = os.path.join(output_base, f"{timestamp}_{method}_seed={seed}_n_step={n}_sigma={s}")
                os.makedirs(output_dir, exist_ok=True)
                reconstruction = dps_save(initial_noise, inverse_problem, eps_net, output_path=output_dir, interval=1)
            if method == "dpms":
                output_dir = os.path.join(output_base, f"{timestamp}_{method}_seed={seed}_n_step={n}_sigma={s}")
                os.makedirs(output_dir, exist_ok=True)
                reconstruction = dpms_save(initial_noise, inverse_problem, eps_net, k, output_path=output_dir, interval=1)
            if method == "dps_dpms":
                output_dir = os.path.join(output_base, f"{timestamp}_{method}_seed={seed}_n_step={n}_sigma={s}_k={k}")
                os.makedirs(output_dir, exist_ok=True)
                reconstruction = dps_dpms_save(initial_noise, inverse_problem, eps_net, k, output_path=output_dir, interval=1)
            




diffusion_pytorch_model.safetensors not found
Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch C:\Users\Dolly\.cache\huggingface\hub\models--google--ddpm-celebahq-256\snapshots\cd5c944777ea2668051904ead6cc120739b86c4d: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\Dolly\.cache\huggingface\hub\models--google--ddpm-celebahq-256\snapshots\cd5c944777ea2668051904ead6cc120739b86c4d.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00,  5.39it/s]
c:\Users\Dolly\.conda\envs\hackathon\lib\site-packages\torch\utils\_device.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
